In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    auc,
    average_precision_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_curve
)
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import tensorflow.keras as keras
from confusion_matrix import plot_confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping  # early stopping
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling1D,
    Input,
    MaxPooling1D
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.regularizers import l1

sns.set()

In [2]:
from pylab import rcParams

# rcParams['figure.figsize'] = 12, 8

In [3]:
RANDOM_SEED = 333
np.random.seed(RANDOM_SEED)

In [4]:
CSV_RESULTS_FOLDER = 'csv_results'
EarlyStoppingCallback = EarlyStopping(monitor='val_loss', patience=5)

# Models

In [ ]:
NUM_CLASSES = 2
SEQ_LEN = 50
NUCLEOTIDES_COUNT = 4
EPOCHS = 10
BATCH_SIZE = 100

In [18]:
model0 = Sequential()
model0.add(Conv1D(50, 9, activation='relu', input_shape=(SEQ_LEN, NUCLEOTIDES_COUNT)))
model0.add(GlobalAveragePooling1D())
model0.add(Dropout(0.3))
model0.add(Dense(NUM_CLASSES, activation='softmax'))

model0.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)
model0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 42, 50)            1850      
_________________________________________________________________
global_average_pooling1d_4 ( (None, 50)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 102       
Total params: 1,952
Trainable params: 1,952
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1 = Sequential()
model1.add(Conv1D(50, 9, activation='relu', input_shape=(SEQ_LEN, NUCLEOTIDES_COUNT)))
model1.add(GlobalAveragePooling1D())
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(NUM_CLASSES, activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)
model1.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 42, 50)            1850      
_________________________________________________________________
global_average_pooling1d (Gl (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                3264      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 5,244
Trainable params: 5,244
Non-trainable params: 0
___________________

In [ ]:
model2 = Sequential()
model2.add(Conv1D(50, 9, activation='relu', padding='same', input_shape=(SEQ_LEN, NUCLEOTIDES_COUNT)))
model2.add(MaxPooling1D(3))
model2.add(Conv1D(50, 3, activation='relu'))
model2.add(GlobalAveragePooling1D())
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(NUM_CLASSES, activation='softmax'))

model2.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 50, 50)            1850      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 16, 50)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 14, 50)            7550      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                3264      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total para

In [ ]:
model3 = Sequential()
model3.add(Conv1D(50, 9, activation='relu', input_shape=(SEQ_LEN, NUCLEOTIDES_COUNT)))
model3.add(MaxPooling1D(3))
model3.add(Conv1D(50, 3, activation='relu'))
model3.add(MaxPooling1D(3))
model3.add(Conv1D(50, 3, activation='relu',))
model3.add(GlobalAveragePooling1D())
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(NUM_CLASSES, activation='softmax'))

model3.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 42, 50)            1850      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 14, 50)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 12, 50)            7550      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 50)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 2, 50)             7550      
_________________________________________________________________
global_average_pooling1d_2 ( (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                3264      
__________

In [19]:
models = {
    '3_CNN_Layers': model3,
    '2_CNN_Layers': model2,
    '1_CNN_Layer': model1,
    '1_CNN_Layer_Simple': model1,
}

## Prepare data

In [ ]:
def one_hot_dna(sequence):
    seq_array = np.array(list(sequence))

    # one hot encoding
    onehot_encoder = OneHotEncoder(sparse=False)
    # reshape because that's what OneHotEncoder likes
    seq_array = seq_array.reshape(len(seq_array), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(seq_array)
    return onehot_encoded_seq

In [ ]:
def read_dataframes(target_pathes, non_target_pathes):

    target_sequences = None
    
    for target_path in target_pathes:
        with open(target_path, 'r') as file:
            read_sequences = np.array([line.strip().upper() for line in file.readlines() if all(base in set(line) for base in 'ACTG')])
            if target_sequences is not None:
                target_sequences = np.concatenate([
                        target_sequences,
                        read_sequences
                    ])
            else:
                target_sequences = read_sequences
            
    non_target_sequences = None
    for non_target_path in non_target_pathes:
        with open(non_target_path, 'r') as file:
            read_sequences = np.array([line.strip().upper() for line in file.readlines() if all(base in set(line) for base in 'ACTG')])
            if non_target_sequences is not None:
                non_target_sequences = np.concatenate([
                        non_target_sequences,
                        read_sequences
                    ]) 
            else:
                non_target_sequences = read_sequences
                                    
    return (target_sequences, non_target_sequences)


In [ ]:
def normalize_datasets(target_sequences, non_target_sequences):
    
    if target_sequences.shape[0] > non_target_sequences.shape[0]:
        target_sequences_n = np.random.choice(
            target_sequences,
            non_target_sequences.shape[0],
        )
        non_target_sequences_n = non_target_sequences
    else:
        target_sequences_n = target_sequences
        non_target_sequences_n = np.random.choice(
            non_target_sequences,
            target_sequences.shape[0],
        )

    X = np.concatenate((target_sequences_n, non_target_sequences_n))
    Y = pd.Series(np.append(
        np.full(target_sequences_n.shape[0], 1),
        np.full(non_target_sequences_n.shape[0], 0))
    )
    
    X = np.array([one_hot_dna(line) for line in X])
    Y = keras.utils.to_categorical(Y)
    
    return (X, Y)


In [ ]:
def compute_metrics(X, Y, model):
    # ROC vars
    tprs = []
    aucs, acc, rec, prec = [], [], [], []
    mean_fpr = np.linspace(0, 1, 100)
    i = 0
    # Precision-recall vars
    precisions = []
    best_precision = {"precision_score": 0.0, "precision": None, "recall": None}

    folded_data = KFold(n_splits=5, random_state=RANDOM_SEED, shuffle=True)

    for k, (train, test) in enumerate(folded_data.split(X, Y)):
        Y_test_flat = np.array(list(map(lambda x: x[1] == 1 and 1 or 0, Y[test])))
        model.fit(
            X[train],
            Y[train],
            # Keras special args
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1,
            validation_split=0.1,
            callbacks=[EarlyStoppingCallback],
            )
        probas_ = model.predict(X[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(Y_test_flat, probas_[:, 1])
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

        Y_pred = model.predict_classes(X[test])

        acc.append(accuracy_score(Y_test_flat, Y_pred))
        prec.append(precision_score(Y_test_flat, Y_pred))
        rec.append(recall_score(Y_test_flat, Y_pred))
        # Compute precision, recall
        precision, recall, _ = precision_recall_curve(Y_test_flat, probas_[:, 1])
        average_precision = average_precision_score(Y_test_flat, probas_[:, 1])
        if average_precision > best_precision["precision_score"]:
            best_precision["precision"] = precision
            best_precision["recall"] = recall
            best_precision["precision_score"] = average_precision
        precisions.append(average_precision)
        
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0

        
    return mean_fpr, mean_tpr, best_precision


In [ ]:
def run_experiment(datasets, model, model_name, write_results=True):
    '''
    datasets: a dict like {
    'L1': 'data_link/L1/data/all_50_last.txt',
    'p_pseudo': 'pseudogenes_50_last.txt',
    }
    model: keras model
    model_name: str
    '''
    # Restart session to ensure that model is clean
    #     keras.backend.clear_session()

    CLASS_1, CLASS_2 = datasets.keys()
    target_pathes, non_target_pathes = datasets.values()
    # Read data
    target_sequences, non_target_sequences = read_dataframes(target_pathes, non_target_pathes)
    X, Y = normalize_datasets(target_sequences, non_target_sequences)

    # Use 5 fold to evaluate model
    mean_fpr, mean_tpr, best_precision = compute_metrics(X, Y, model)

    if write_results:
        # Create file names
        CSV_FILE_SUBNAME_OBJECTS = f"{CLASS_1}_vs_{CLASS_2}" # "True_vs_False"
        CSV_FILE_SUBNAME = f"{CSV_FILE_SUBNAME_OBJECTS}__{model_name}"

        # Write results to csv
        # ROC
        pd.DataFrame({
            "fpr": mean_fpr,
            "tpr": mean_tpr
        }).to_csv(
            f"{CSV_RESULTS_FOLDER}/ROC__{CSV_FILE_SUBNAME}.csv",
            index=False
        )
        # Precision-recall
        pd.DataFrame({
            "precision": best_precision["precision"],
            "recall": best_precision["recall"]
        }).to_csv(
            f"{CSV_RESULTS_FOLDER}/Precision-Recall__{CSV_FILE_SUBNAME}.csv",
            index=False
        )
        #     pd.DataFrame(fi).to_csv("Feature_importance__{0}.csv".format(CSV_FILE_SUBNAME), index=False)
    return model







# Experiments

In [ ]:
experiments = [
    {
    'L1': ['data_link/L1/data/all_50_last.txt',],
    'p_pseudo': ['pseudogenes_50_last.txt',],
    },
    {
    'L1': ['data_link/L1/data/all_50_last.txt',],
    'mRNA': ['KnownGene_50_last.txt',],
    },
    {
    'L1_p_pseudo': ['data_link/L1/data/all_50_last.txt', 'pseudogenes_50_last.txt',],
    'shuffled': ['data_link/L1/data/all_shuffled_50_last.txt', 'pseudogenes_50_last_shuffled.txt',],
    },
    {
    'L1_mRNA': ['data_link/L1/data/all_50_last.txt', 'KnownGene_50_last.txt',],
    'shuffled': ['data_link/L1/data/all_shuffled_50_last.txt', 'KnownGene_50_last_shuffled.txt',],
    },
    {
    'mRNA': ['KnownGene_50_last.txt',],
    'shuffled': ['KnownGene_50_last_shuffled.txt',],
    },
    {
    'p_pseudo': ['pseudogenes_50_last.txt',],
    'shuffled': ['pseudogenes_50_last_shuffled.txt',],
    },
]

In [20]:
for model_name, model in models.items():
    for experiment_dataset in experiments:
        run_experiment(experiment_dataset, model, model_name)

67745/67745 [==============================] - 2s 26us/sample - loss: 0.4440 - acc: 0.7856 - val_loss: 0.5207 - val_acc: 0.7582
Epoch 5/10
67745/67745 [==============================] - 2s 25us/sample - loss: 0.4432 - acc: 0.7871 - val_loss: 0.5429 - val_acc: 0.7306
Epoch 6/10
67745/67745 [==============================] - 2s 25us/sample - loss: 0.4424 - acc: 0.7873 - val_loss: 0.4704 - val_acc: 0.7965
Epoch 7/10
67745/67745 [==============================] - 2s 25us/sample - loss: 0.4418 - acc: 0.7890 - val_loss: 0.5270 - val_acc: 0.7426
Epoch 8/10
67745/67745 [==============================] - 2s 25us/sample - loss: 0.4403 - acc: 0.7897 - val_loss: 0.4600 - val_acc: 0.8017
Train on 67745 samples, validate on 7528 samples
Epoch 1/10
67745/67745 [==============================] - 2s 26us/sample - loss: 0.4446 - acc: 0.7849 - val_loss: 0.4667 - val_acc: 0.7790
Epoch 2/10
67745/67745 [==============================] - 2s 26us/sample - loss: 0.4430 - acc: 0.7850 - val_loss: 0.5082 - val_a